# Bitcoin price forecasting with PySpark
## Big Data Computing final project - A.Y. 2022 - 2023
Prof. Gabriele Tolomei

MSc in Computer Science

La Sapienza, University of Rome

### Author
Corsi Danilo - corsi.1742375@studenti.uniroma1.it



Description: In this notebook I am going to explore the data and visualize the correlations check their stationarity and choose features to use to train the models.

# Dependencies, Libraries and Tools

In [30]:
JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"
SLOW_OPERATION = True

In [31]:
#Install some useful dependencies
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from itertools import cycle

import plotly.express as px

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import gc

import pandas as pd
import numpy as np
import json

import matplotlib.pyplot as plt
import seaborn as sns

# !pip install -U -q PyDrive # To use files that are stored in Google Drive directly (e.g., without downloading them from an external URL)
# !apt install openjdk-8-jdk-headless -qq
# import os
# os.environ["JAVA_HOME"] = JAVA_HOME

# Link to Google Drive

In [32]:
GDRIVE_DIR = "/content/drive"

GDRIVE_DATASET_RAW_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/datasets/raw"
GDRIVE_DATASET_TEMP_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/datasets/temp"
GDRIVE_DATASET_OUTPUT_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/datasets/output"

GDRIVE_DATASET_NAME = "bitcoin_blockchain_data_1h"

GDRIVE_DATASET_NAME_EXT = "/" + GDRIVE_DATASET_NAME + ".parquet"

GDRIVE_DATASET = GDRIVE_DATASET_RAW_DIR + GDRIVE_DATASET_NAME_EXT

SLOW_OPERATION: False

In [33]:
# Point Colaboratory to our Google Drive
from google.colab import drive

drive.mount(GDRIVE_DIR, force_remount=True)

Mounted at /content/drive


# Data visualization ❗

In this section I'm going to explore the dataset
First, we import the dataset from Google Drive, check the shape and print out the schema.

In [34]:
df = pd.read_parquet(GDRIVE_DATASET)
df

,market-price,market-cap,total-bitcoins,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd
timestamp,,,,,,,,,,,,,,,
2016-01-01 00:00:00,430.890000,6.474140e+09,1.502958e+07,2.860153e+06,54604.791735,0.493407,1.011557e+08,919.200000,6.971292e+05,1.038803e+11,1.554769e+06,8783.063851,263121.000000,124092.000000,5.834626e+07
2016-01-01 01:00:00,431.050833,6.475204e+09,1.502981e+07,2.809565e+06,54607.538237,0.497004,1.011608e+08,923.727011,6.975595e+05,1.038803e+11,1.559629e+06,8992.039015,266036.875000,125131.416667,5.803318e+07
2016-01-01 02:00:00,431.211667,6.476268e+09,1.503004e+07,2.758977e+06,54610.284739,0.500602,1.011659e+08,928.254023,6.979898e+05,1.038803e+11,1.564490e+06,9201.014178,268952.750000,126170.833333,5.772010e+07
2016-01-01 03:00:00,431.372500,6.477333e+09,1.503027e+07,2.708389e+06,54613.031241,0.504199,1.011710e+08,932.781034,6.984201e+05,1.038803e+11,1.569350e+06,9409.989342,271868.625000,127210.250000,5.740702e+07
2016-01-01 04:00:00,431.533333,6.478397e+09,1.503050e+07,2.657801e+06,54615.777742,0.507797,1.011761e+08,937.308046,6.988505e+05,1.038803e+11,1.574211e+06,9618.964505,274784.500000,128249.666667,5.709394e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-29 20:00:00,29352.595000,5.688077e+11,1.944331e+07,3.926979e+07,499595.925800,1.704247,8.715501e+08,3737.882035,4.214032e+08,5.232831e+13,3.052827e+07,503742.351307,782832.666667,608731.500000,1.421208e+09
2023-07-29 21:00:00,29354.418750,5.687328e+11,1.944336e+07,3.743955e+07,499604.814792,1.708625,8.715677e+08,3764.503788,4.253051e+08,5.232831e+13,3.081936e+07,505222.084235,787249.500000,618012.125000,1.412168e+09
2023-07-29 22:00:00,29356.242500,5.686580e+11,1.944340e+07,3.560932e+07,499613.703783,1.713002,8.715854e+08,3791.125541,4.292070e+08,5.232831e+13,3.111045e+07,506701.817164,791666.333333,627292.750000,1.403129e+09


In [35]:
df.shape

(66409, 15)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 66409 entries, 2016-01-01 00:00:00 to 2023-07-30 00:00:00
Data columns (total 15 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   market-price                      66409 non-null  float64
 1   market-cap                        66409 non-null  float64
 2   total-bitcoins                    66409 non-null  float64
 3   trade-volume                      66409 non-null  float64
 4   blocks-size                       66409 non-null  float64
 5   avg-block-size                    66409 non-null  float64
 6   n-transactions-total              66409 non-null  float64
 7   n-transactions-per-block          66409 non-null  float64
 8   hash-rate                         66409 non-null  float64
 9   difficulty                        66409 non-null  float64
 10  miners-revenue                    66409 non-null  float64
 11  transaction-fees-usd            

In [37]:
def show_metrics(dataset, feature):
  trace = go.Scatter(
      x = dataset.index,
      y = dataset[feature].astype(float),
      mode = 'lines',
      name = feature
  )

  layout = dict(
      title=feature,
      xaxis=dict(
          rangeselector=dict(
              buttons=list([
                  #change the count to desired amount of months.
                  dict(count=1,
                      label='1m',
                      step='month',
                      stepmode='backward'),
                  dict(count=6,
                      label='6m',
                      step='month',
                      stepmode='backward'),
                  dict(count=12,
                      label='1y',
                      step='month',
                      stepmode='backward'),
                  dict(count=36,
                      label='3y',
                      step='month',
                      stepmode='backward'),
                  dict(step='all')
              ])
          ),
          rangeslider=dict(
              visible = True
          ),
          type='date'
      )
  )

  data = [trace]
  fig = dict(data=data, layout=layout)
  iplot(fig, filename = "Time Series with Rangeslider")

In [38]:
currency_statistics = ['market-price', 'market-cap', 'total-bitcoins', 'trade-volume']
block_details = ['blocks-size', 'avg-block-size', 'n-transactions-total', 'n-transactions-per-block']
mining_information = ['hash-rate', 'difficulty', 'miners-revenue', 'transaction-fees-usd']
network_activity = ['n-unique-addresses', 'n-transactions', 'estimated-transaction-volume-usd']

In [39]:
if SLOW_OPERATION:
  for feature in currency_statistics:
    show_metrics(df, feature)

Output hidden; open in https://colab.research.google.com to view.

In [40]:
if SLOW_OPERATION:
  for feature in block_details:
    show_metrics(df, feature)

Output hidden; open in https://colab.research.google.com to view.

In [41]:
if SLOW_OPERATION:
  for feature in mining_information:
    show_metrics(df, feature)

Output hidden; open in https://colab.research.google.com to view.

In [42]:
if SLOW_OPERATION:
  for feature in network_activity:
    show_metrics(df, feature)

Output hidden; open in https://colab.research.google.com to view.

# Checking stationarity❗

Source: https://www.kaggle.com/code/debashis74017/time-series-forecasting-itcoin-price?scriptVersionId=113747601&cellId=25

Stationarity means that the statistical properties of a time series i.e. mean, variance and covariance do not change over time. Many statistical models require the series to be stationary to make effective and precise predictions.

Two statistical tests would be used to check the stationarity of a time series:
* Augmented Dickey Fuller (“ADF”) test
* Kwiatkowski-Phillips-Schmidt-Shin (“KPSS”) test.

## ADF Test ❗
ADF test is used to determine the presence of unit root in the series, and hence helps in understand if the series is stationary or not. The null and alternate hypothesis of this test are:

* Null Hypothesis: The series has a unit root.

* Alternate Hypothesis: The series has no unit root.

If the null hypothesis in failed to be rejected, this test may provide evidence that the series is non-stationary.

In [43]:
from statsmodels.tsa.stattools import adfuller

if SLOW_OPERATION:
  result = adfuller(df['market-price'], autolag='AIC')
  print(f'ADF Statistic: {result[0]}')
  print(f'p-value: {result[1]}')
  for key, value in result[4].items():
      print('Critial Values:')
      print(f'   {key}, {value}')

ADF Statistic: -1.4234143441085858
p-value: 0.5710009478621985
Critial Values:
   1%, -3.430448565918899
Critial Values:
   5%, -2.861583564348427
Critial Values:
   10%, -2.5667931878206702


ADF Stats value is greater than all critical values, and p-value is also greater than 0.05. So we can strongly reject the null hypothesis, and conclude that, Price value is Non-Stationary.

Let's apply log transformation to the data and test again.

In [44]:
from numpy import log

if SLOW_OPERATION:
  result = adfuller((log(df['market-price'])), autolag='AIC')
  print(f'ADF Statistic: {result[0]}')
  print(f'p-value: {result[1]}')
  for key, value in result[4].items():
      print('Critial Values:')
      print(f'   {key}, {value}')

ADF Statistic: -1.8889379078655046
p-value: 0.3372828286401903
Critial Values:
   1%, -3.430448565918899
Critial Values:
   5%, -2.861583564348427
Critial Values:
   10%, -2.5667931878206702


After applying Log transformation also, ADF Stats value is greater than all critical values, and p-value is also greater than 0.05. It seems, Price value is purely Non-Stationary.

## KPSS test - Kwiatkowski Phillips Schmidt Shin ❗
KPSS is another test for checking the stationarity of a time series. The null and alternate hypothesis for the KPSS test are opposite that of the ADF test:

* Null Hypothesis: The process is trend stationary.

* Alternate Hypothesis: The series has a unit root (series is not stationary).

In [45]:
from statsmodels.tsa.stattools import kpss

if SLOW_OPERATION:
  result = kpss(df['market-price'], regression='c')
  print('\nKPSS Statistic: %f' % result[0])
  print('p-value: %f' % result[1])
  for key, value in result[3].items():
      print('Critial Values:')
      print(f'   {key}, {value}');


KPSS Statistic: 26.943189
p-value: 0.010000
Critial Values:
   10%, 0.347
Critial Values:
   5%, 0.463
Critial Values:
   2.5%, 0.574
Critial Values:
   1%, 0.739


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/stattools.py:2018: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.




The output of the KPSS test contains 4 values:

* **The KPSS statistic**: is the actual test statistic that is computed while performing the test.
* **p-value**: is the probability score based on which you can decide whether to reject the null hypothesis or not. If the p-value is less than a predefined alpha level (typically 0.05), we reject the null hypothesis.
* **Number of lags used by the test**: is the number of lags of the series that was actually used by the model equation of the kpss test. By default, the statsmodels kpss() uses the ‘legacy’ method. In legacy method, int(12 * (n / 100)**(1 / 4)) number of lags is included, where n is the length of the series.
* **Critical values**: in order to reject the null hypothesis, the test statistic should be greater than the provided critical values. If it is in fact higher than the target critical value, then that should automatically reflect in a low p-value. If the p-value is less than 0.05, the kpss statistic will be greater than the 5% critical value.

Here we find that, KPSS stats value is too high than critical values, so, we concluded that this time series is Non-Stationary

#  Feature selection ❗

In [54]:
# For quick access
all_columns = df.columns
print(all_columns)

# Continuous
cont_columns = df.columns[1:]
print(cont_columns)

# Dependent Var
dep_var = 'market-price'

Index(['market-price', 'market-cap', 'total-bitcoins', 'trade-volume',
       'blocks-size', 'avg-block-size', 'n-transactions-total',
       'n-transactions-per-block', 'hash-rate', 'difficulty', 'miners-revenue',
       'transaction-fees-usd', 'n-unique-addresses', 'n-transactions',
       'estimated-transaction-volume-usd'],
      dtype='object')
Index(['market-cap', 'total-bitcoins', 'trade-volume', 'blocks-size',
       'avg-block-size', 'n-transactions-total', 'n-transactions-per-block',
       'hash-rate', 'difficulty', 'miners-revenue', 'transaction-fees-usd',
       'n-unique-addresses', 'n-transactions',
       'estimated-transaction-volume-usd'],
      dtype='object')


In [55]:
# Compute the correlation matrix
matrix_pearson = df.corr(method='pearson')
matrix_spearman = df.corr(method='spearman')

# Sorting values
pearson_correlations = matrix_pearson[dep_var].sort_values(ascending=False)
pearson_correlations = pearson_correlations[pearson_correlations.index != dep_var]
spearman_correlations = matrix_spearman[dep_var].sort_values(ascending=False)
spearman_correlations = spearman_correlations[spearman_correlations.index != dep_var]

# Compute the mean between the two correlation matrix
features = {}
for key in pearson_correlations.keys():
    features[key] = [pearson_correlations[key], spearman_correlations[key]]

mean_features = {}
for key in features.keys():
    mean_features[key] = sum(features[key]) / len(features[key])

cor_matrix_features =  dict(list(mean_features.items())[:7])
print(cor_matrix_features)

{'market-cap': 0.9993217398099414, 'miners-revenue': 0.9449609710592767, 'estimated-transaction-volume-usd': 0.8727636713116008, 'n-transactions-total': 0.8090676930183996, 'blocks-size': 0.8052757794537508, 'total-bitcoins': 0.7948017405775403, 'n-unique-addresses': 0.6963215727346038}


In [56]:
from sklearn.ensemble import GradientBoostingRegressor

# Separate features and target variable
X = df.drop(columns=[dep_var])
y = df[dep_var]

# Train the model with the optimal parameters
gb = GradientBoostingRegressor(n_estimators=100, max_depth=5, learning_rate=0.05)

# Train the model on the features and the target variable
gb.fit(X, y)

# Get the most important features
importances = gb.feature_importances_
sorted_indices = importances.argsort()[::-1]

features = {}
for i in sorted_indices:
    features[X.columns[i]] = importances[i]

gb_features =  dict(list(features.items())[:7])
print(gb_features)

{'market-cap': 0.9982576804435397, 'blocks-size': 0.0004688408414286752, 'miners-revenue': 0.0003663856456549574, 'total-bitcoins': 0.0002797116990342391, 'difficulty': 0.00017464223612113333, 'n-transactions-total': 0.00012826747163049643, 'estimated-transaction-volume-usd': 0.00011827297983739706}


#  [OLD] Feature selection ❗

Source: https://medium.com/experimenting-with-deep-learning/predictbit-d0c1e44990a3


The aim of this section is to remove the features that have less of an impact to the final result from the dataset. This operation is called feature selection.

In [46]:
# # For quick access
# all_columns = df.columns
# print(all_columns)

# # Continuous
# cont_columns = df.columns[1:]
# print(cont_columns)

# # Dependent Var
# dep_var = 'market-price'

Plotting a correlation heatmap for Market Price (our target) against all the continuous features yields the following:

In [47]:
# cor = df.corr().abs()

In [48]:
# # Correlation b/w Market Price and Continuous Features:
# cont_df = df[cont_columns]
# cont_df[dep_var] = df[dep_var]
# cont_corr_df = cont_df.corr()
# plt.figure(figsize=(15,1))
# sns.heatmap(cont_corr_df.sort_values(by=dep_var, axis=1).drop(dep_var, axis=1).tail(1), annot=True, cmap=plt.cm.Reds)
# plt.show()

Before we eliminate any columns, lets first filter out and keep features that satisfy this first condition: (1) highly correlated to our target.

We can then visualize this filtered subset with a Seaborn heat map to inductively identify which features are also correlated to each other.

To get this to work, first, we’ll collect all correlation comparisons to our target in a variable called “cor_target”.

In [49]:
# cor_target = cor[dep_var]

In [50]:
# # Identifying Features Highly Correlated with Target and Other Features:
# relevant_features = cor_target[cor_target>0.6]

# rel_df = relevant_features.to_frame() #.transpose()
# rel_df = rel_df.sort_values(dep_var, ascending=False)
# rel_columns = rel_df[1:].index.values.tolist()
# rel_corr_df = df[rel_columns].corr()

# plt.figure(figsize=(12,10))
# mask = np.zeros_like(rel_corr_df, dtype=np.bool)
# mask[np.triu_indices_from(mask)] = True
# sns.heatmap(rel_corr_df[rel_corr_df > .6], annot=True, cmap=plt.cm.Reds, mask=mask)
# rel_columns, plt.show()

The above heat map illustration demonstrates that several of these filtered features are also correlated to each other.

In this scenario, we want to minimize the number of red cells by, again, filtering by a certain threshold.

I used a higher threshold, here, to prevent dropping too many features.

In [51]:
# # Reducing Dimensionality of Correlation Matrix:
# def dimentionalize_corr_matrix(cor_df, upper_bound=0.95):
#   upper = cor_df.where(np.triu(np.ones(cor_df.shape), k=1).astype(np.bool))
#   to_drop = [column for column in upper.columns if any(upper[column] > upper_bound)]
#   rel_df_clean = cor_df.drop(cor_df[to_drop], axis=0)
#   rel_df_clean =  rel_df_clean.drop( rel_df_clean[to_drop], axis=1)
#   return rel_df_clean


# rel_df_clean = dimentionalize_corr_matrix(rel_corr_df, .9)
# rel_columns = rel_df_clean.columns.tolist()

# plt.figure(figsize=(12,10))
# mask = np.zeros_like(rel_df_clean, dtype=np.bool)
# mask[np.triu_indices_from(mask)] = True
# sns.heatmap(rel_df_clean, annot=True, cmap=plt.cm.Reds, mask=mask)
# rel_columns, plt.show()

The outputted features are: 'market-cap', 'estimated-transaction-volume-usd', 'blocks-size', 'n-unique-addresses'.

We’ll call these columns “relevant columns”.

I decided to use asecond technique for feature elimination from the sklearn library, the RFE method recursively goes through the feature set, removing columns to get the best accuracy level it deems possible.

To use RFE, we first have to declare a sklearn linear regression model (not to be confused with the pyspark Linear Regression model we’ll be using in next steps) and pass that model to the RFE method with the maximum number of features we’d like to extract.

We’re looking for 7 recommended features (the same list length as “relevant columns”) from the RFE model to create a second test group of features. To use the RFE method, we also have to get a list of targets to support RFE’s accuracy check:

In [52]:
# from sklearn.feature_selection import RFE
# from sklearn.linear_model import LinearRegression as sklearnLR

# model = sklearnLR()
# rfe = RFE(model, step = 7)
# y = df[dep_var].tolist()

We fit the RFE model on the Sklearn Linear Regression model, along with the inputs and targets:

In [53]:
# X_rfe = rfe.fit_transform(df.drop(dep_var, axis=1), y)
# model.fit(X_rfe, y)
# temp = pd.Series(rfe.support_,index = df.drop(dep_var, axis=1).columns)
# selected_features_rfe = temp[temp==True].index.values.tolist()
# print(selected_features_rfe)

The outputted features: 'total-bitcoins', 'blocks-size', 'avg-block-size', 'n-transactions-per-block', 'miners-revenue', 'n-unique-addresses', 'n-transactions'.

Vastly different from the list generated earlier! Let’s see how it goes.

# Output

In the latter section I'm going to save the features I extrapolated previously.

In [59]:
GDRIVE_FEATURES_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/features"

GDRIVE_COR_MATRIX_FEATURES_NAME = "cor_matrix_features"
GDRIVE_GB_FEATURES_NAME = "gb_features"

GDRIVE_COR_MATRIX_FEATURES_NAME_EXT = "/" + GDRIVE_COR_MATRIX_FEATURES_NAME + ".json"
GDRIVE_GB_FEATURES_NAME_EXT = "/" + GDRIVE_GB_FEATURES_NAME + ".json"

GDRIVE_COR_MATRIX_FEATURES = GDRIVE_FEATURES_DIR + GDRIVE_COR_MATRIX_FEATURES_NAME_EXT
GDRIVE_GB_FEATURES = GDRIVE_FEATURES_DIR + GDRIVE_GB_FEATURES_NAME_EXT

In [60]:
with open(GDRIVE_COR_MATRIX_FEATURES, 'w') as file:
    json.dump(cor_matrix_features, file)

In [61]:
with open(GDRIVE_GB_FEATURES, 'w') as file:
    json.dump(gb_features, file)